<a href="https://colab.research.google.com/github/Jee-9/Study/blob/main/Preprocessing_ResNet101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import os 
from os.path import join

"""## Pytorch를 이용한 이미지 데이터 가져오기"""

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader 
import torchvision
from torchvision import transforms

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Base Path 설정

base_dir = "/content/drive/MyDrive/PROJECT SHARING FILES"
folder  = "imageData"
cats = ["education_images", "game_images", "kpop_images", "mukbang_images"]

for c in cats:
  vars()["path_" + c] = join(base_dir, folder, c)

root_path = join(base_dir, folder)

In [4]:
"""- 일부 이미지 augmentation ( GaussianBlur )"""

blur_trans = transforms.RandomApply([
  transforms.GaussianBlur(15)
],  p = 0.02)

"""- 일부 이미지 Gaussian Noise 추가하기"""

class AddGaussianNoise(object):
  def __init__(self, mean = 0.0, std = 1.0):
    self.mean = mean
    self.std = std
  
  def __call__(self, tensor):
    return tensor+torch.randn(tensor.size())*self.std+self.mean

  def __repr__(self):
    return self.__class__.__name__+'(mean={0}, std={1})'.format(self.mean, self,std)

noise_trans = transforms.RandomApply([                                   
  AddGaussianNoise()
], p = 0.02)

"""- 최종 transforms"""

fin_trans = transforms.Compose([transforms.Resize((244,244)), 
                            blur_trans,
                            noise_trans,
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
                            ])

In [44]:
trainset = torchvision.datasets.ImageFolder(root = root_path,
                                            transform = fin_trans)

In [36]:
# trainloader = DataLoader(trainset, 
#                          batch_size = 512,
#                          shuffle = True,
#                          num_workers = 2)

# Model

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch import optim
from torch.optim.lr_scheduler import StepLR

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os

from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

import time
import copy
import numpy as np

In [18]:
class BottleNeck(nn.Module):
  expansion = 4
  def __init__(self, in_channels, out_channels, stride=1):
    super().__init__()

    self.residual_function = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False), 
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False), 
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, stride=1, bias=False),
        nn.BatchNorm2d(out_channels * BottleNeck.expansion), 
    )

    self.shortcut = nn.Sequential()
    self.relu = nn.ReLU()

    if stride != 1 or in_channels != out_channels * BottleNeck.expansion: 
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_channels, out_channels * BottleNeck.expansion, kernel_size=1, stride=stride, bias=False),
          nn.BatchNorm2d(out_channels*BottleNeck.expansion) 
      ) 

  def forward(self, x):
    x = self.residual_function(x) + self.shortcut(x)
    x = self.relu(x)
    return x

In [19]:
class ResNet(nn.Module):
  def __init__(self, block, num_block, num_classes=4, init_weights = True):

    super().__init__()

    self.in_channels=64

    self.conv1 = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

    # Stacking layers
    self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
    self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
    self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
    self.conv5_x = self._make_layer(block, 512, num_block[3], 2)

    self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512*block.expansion, num_classes)

    if init_weights:
      self._initialize_weights()

  def _make_layer(self, block, out_channels, num_blocks, stride):
    strides = [stride] + [1] * (num_blocks-1)
    layers=[]
    for stride in strides:
      layers.append(block(self.in_channels, out_channels, stride))
      self.in_channels = out_channels * block.expansion
    return nn.Sequential(*layers)

  def forward(self, x):
    f_output = self.conv1(x)
    output = self.conv2_x(f_output)
    x = self.conv3_x(output)
    x = self.conv4_x(x)
    x = self.conv5_x(x)
    x = self.avg_pool(x)
    x = x.view(x.size(0), -1)

    x = self.fc(x)
    return x

  def _initialize_weights(self):
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode = 'fan_out', nonlinearity='relu')
        if m.bias is not None:
          nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.constant_(m.bias, 0)


def resnet50():
  return ResNet(BottleNeck, [3,4,6,3])

def resnet101():
  return ResNet(BottleNeck, [3,4,23,3])

def resnet152():
  return ResNet(BottleNeck, [3,8,36,3])

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = resnet101().to(device)
x = torch.randn(3,3,224,224).to(device)
output = model(x)
print(output.size())

torch.Size([3, 4])


In [21]:
summary(model, (3,224,224), device = device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

# 학습용 함수들

In [27]:
# 손실함수, optimizer, lr_scheduler 정의
loss_func = nn.CrossEntropyLoss(reduction='sum')
opt = optim.Adam(model.parameters(), lr=0.001)

from torch.optim.lr_scheduler import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(opt, mode='min', factor=0.1, patience=10)

# 현재 lr 계산하는 함수
def get_lr(opt):
  for param_group in opt.param_groups:
    return param_group['lr']

# 배치당 loss와 metric을 계산하는 함수
def metric_per_batch(output, target):
  pred = output.argmax(1, keepdim=True)
  corrects = pred.eq(target.view_as(pred)).sum().item()
  return corrects

def loss_per_batch(loss_func, output, target, opt=None):
  loss = loss_func(output, target)
  metric_b = metric_per_batch(output, target)

  if opt is not None:
    opt.zero_grad()
    loss.backward()
    opt.step()

  return loss.item(), metric_b

# 에폭당 loss를 정의하는 함수
def loss_per_epoch(model, loss_func, trainloader, sanity_check=False, opt=None):
  running_loss = 0.0
  running_metric = 0.0
  len_data = len(trainloader.dataset)

  for xb, yb in trainloader:
    xb = xb.to(device)
    yb = yb.to(device)
    output = model(xb)

    loss_b, metric_b = loss_per_batch(loss_func, output, yb, opt)

    running_loss += loss_b

    if metric_b is not None:
      running_metric += metric_b

    if sanity_check is True:
      break
  loss = running_loss / len_data
  metric = running_metric / len_data

  return loss, metric

# 학습함수

In [31]:
def train_model(model, params):

  num_epochs = params['num_epochs']
  loss_func = params['loss_func']
  opt = params['optimizer']
  trainloader = params['trainloader']
  # validationlader = params['validloader']
  sanity_check = params['sanity_check']
  lr_scheduler = params['lr_scheduler']
  path2weights = params['path2weights']

  loss_history = {'train': [], 'val' : []}
  metric_history = {'train': [], 'val': []}

  best_loss = float('inf')

  start_time = time.time()

  for epoch in range(num_epochs):
    current_lr = get_lr(opt)
    print('Epoch {}/{}, current lr = {}'.format(epoch, num_epochs-1, current_lr))

    model.train()
    print('model training finished')

    # train_loss, train_metric = loss_per_epoch(model, loss_func, train_dl, sanity_check, opt)
    # print('loss_per_epoch finished') 
    # loss_history['train'].append(train_loss)
    # metric_history['train'].append(train_metric)

    # model.eval()
    # print('model.eval() finished') 
    # with torch.no_grad():
    #   val_loss, val_metric = loss_per_epoch(model, loss_func, validloader, sanity_check)
    # loss_history['val'].append(val_loss)
    # metric_history['val'].append(val_metric)

    # if val_loss < best_loss:
    #   best_loss = val_loss
    #   # best_model_wts = copy.deepcopy(model.state_dict())

    #   # torch.save(model.state_dict(), path2weights)
    #   # print('Copied best model weights!')
    #   print('Got best val_loss')

    # lr_scheduler.step(val_loss)
    # # wandb.log({'Epoch': epoch, 'loss': np.mean(loss_arr)}) # 추가된 코드 3
    # print('train loss: %.6f, val loss: %.6f, accuracy: %.2f, time: %.4f min' %(train_loss, val_loss, 100*val_metric, (time.time()-start_time)/60))
    # print('-'*10)

  return model
  # , loss_history, metric_history

In [69]:
# 하이퍼 파라미터 정의

params_train = {
    'num_epochs': 90, # 이거 첨에 1000으로 줬는데 아마 오버피팅되었을 가능성이,, ㅎ
    'optimizer' : opt,
    'loss_func' : loss_func,
    'trainloader' : trainloader,
    'validloader' : validloader,
    'sanity_check' : False,
    'lr_scheduler' : lr_scheduler,
    'path2weights' : './models/weights.pt',
}

def createFolder(directory):
  try:
    if not os.path.exists(directory):
      os.makedirs(directory)
  except OSerror:
    print('Error')

createFolder('./models')

In [68]:
# model, loss_hist, metric_hist = train_val(model, params_train)

model = train_model(model, params_train)

Epoch 0/89, current lr = 0.001
model training finished
Epoch 1/89, current lr = 0.001
model training finished
Epoch 2/89, current lr = 0.001
model training finished
Epoch 3/89, current lr = 0.001
model training finished
Epoch 4/89, current lr = 0.001
model training finished
Epoch 5/89, current lr = 0.001
model training finished
Epoch 6/89, current lr = 0.001
model training finished
Epoch 7/89, current lr = 0.001
model training finished
Epoch 8/89, current lr = 0.001
model training finished
Epoch 9/89, current lr = 0.001
model training finished
Epoch 10/89, current lr = 0.001
model training finished
Epoch 11/89, current lr = 0.001
model training finished
Epoch 12/89, current lr = 0.001
model training finished
Epoch 13/89, current lr = 0.001
model training finished
Epoch 14/89, current lr = 0.001
model training finished
Epoch 15/89, current lr = 0.001
model training finished
Epoch 16/89, current lr = 0.001
model training finished
Epoch 17/89, current lr = 0.001
model training finished
Ep

# 모델 저장

In [40]:
# Google drive 에 save
model_save_name = 'ResNet101.pt'
path = F"/content/drive/MyDrive/{model_save_name}"
torch.save(model.state_dict(), path)

In [ ]:
# Google drive에서 load
model_save_name = 'ResNet101.pt'
path = F"/content/drive/MyDrive/{model_save_name}"
torch.load_state_dict(torch.load(path))

# train_test_split in image

In [57]:
# 1번 방법
train_size = int(0.8*len(trainset))
test_size = len(trainset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(trainset, [train_size, test_size])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=True)
validloader = torch.utils.data.DataLoader(test_dataset, batch_size=512, shuffle=False)

In [60]:
len(trainloader) # 512*78 = 3만9천9백개 = 거의 4만개

78

In [61]:
len(validloader) # 512*20 = 1만240개 = 1만개 // 도합 5만개 ㅇㅋ

20

In [62]:
# 2번 방법
from torch.utils.data.sampler import SubsetRandomSampler

batch_size = 512
validation_split = .2
shuffle_dataset = True
random_seed = 42

# training, test 분할을 위해 인덱스 생성
dataset_size = len(trainset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split*dataset_size))

if shuffle_dataset:
  np.random.seed(random_seed)
  np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
validloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, sampler=valid_sampler)

In [63]:
len(trainloader)

78

In [64]:
len(validloader)

20

In [77]:
next(iter(trainloader)) # next만 붙으면 에러가 뜬다.

IndexError: ignored

In [75]:
# 확인하는 방법이라는데 work XXX

inputs, classes = next(iter(trainloader))
print(classes)

def imshow(inp, title=None):
  inpt = inp.numpy().transpose((1,2,0))
  mean = np.array([0.5,0.5,0.5])
  std = np.array([0.5,0.5,0.5])
  inp = std*inp+mean
  inp = np.clip(inp, 0 ,1)
  plt.imshow(inp)
  if title is not None:
    plt.title(title)
  plt.pause(0.001)

out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

IndexError: ignored

# 이제 split 했으니까 한번 돌려보자

In [66]:
model.eval()

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (conv2_x): Sequential(
    (0): BottleNeck(
      (residual_function): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
        (6): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential(
     

In [73]:
def train_model(model, params):

  num_epochs = params['num_epochs']
  loss_func = params['loss_func']
  opt = params['optimizer']
  trainloader = params['trainloader']
  validloader = params['validloader']
  sanity_check = params['sanity_check']
  lr_scheduler = params['lr_scheduler']
  path2weights = params['path2weights']

  loss_history = {'train': [], 'val' : []}
  metric_history = {'train': [], 'val': []}

  best_loss = float('inf')

  start_time = time.time()

  for epoch in range(num_epochs):
    current_lr = get_lr(opt)
    print('Epoch {}/{}, current lr = {}'.format(epoch, num_epochs-1, current_lr))

    model.train()
    print('model training finished')

    train_loss, train_metric = loss_per_epoch(model, loss_func, trainloader, sanity_check, opt)
    print('loss_per_epoch finished') 
    loss_history['train'].append(train_loss)
    metric_history['train'].append(train_metric)

    model.eval()
    print('model.eval() finished') 
    with torch.no_grad():
      val_loss, val_metric = loss_per_epoch(model, loss_func, validloader, sanity_check)
    loss_history['val'].append(val_loss)
    metric_history['val'].append(val_metric)

    if val_loss < best_loss:
      best_loss = val_loss
      # best_model_wts = copy.deepcopy(model.state_dict())

      # torch.save(model.state_dict(), path2weights)
      # print('Copied best model weights!')
      print('Got best val_loss')

    lr_scheduler.step(val_loss)
    # wandb.log({'Epoch': epoch, 'loss': np.mean(loss_arr)}) # 추가된 코드 3
    print('train loss: %.6f, val loss: %.6f, accuracy: %.2f, time: %.4f min' %(train_loss, val_loss, 100*val_metric, (time.time()-start_time)/60))
    print('-'*10)

  return model, loss_history, metric_history

In [74]:
model, loss_hist, metric_hist = train_model(model, params_train)

Epoch 0/89, current lr = 0.001
model training finished


IndexError: ignored